In [148]:
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
 
options = Options()
options.add_argument('--headless')
options.add_argument('--disable-gpu')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

browser = webdriver.Chrome(
    executable_path='./chromedriver_win32/chromedriver.exe',options=options)

browser.get('https://stratus.unisa.ac.za/sacdl20/asps/unisa_assignments.asp')

studentNumber = browser.find_element(By.NAME, 'StudNo')
studentNumber.send_keys('REDACTED')

studentPassword = browser.find_element(By.NAME, 'password')
studentPassword.send_keys('REDACTED')

submit = browser.find_element(By.CSS_SELECTOR, 'input[type="submit"]')
submit.click()

# then redirect to https://stratus.unisa.ac.za/sacdl20/asps/stuasstab.asp?system=SOL
# wait for the page to load
redirectURL = 'https://stratus.unisa.ac.za/sacdl20/asps/stuasstab.asp?system=SOL'

browser.get(redirectURL)

# get the html of the page
html = browser.page_source

browser.quit()

C:\Users\ADGSENPAI\AppData\Local\Temp\ipykernel_16948\1474098441.py:14: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(


In [149]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(html, 'html.parser')

# find the table
table = soup.find('table')

In [150]:
textBuffer = ''
for row in table.find_all('tr'):
    for cell in row.find_all('td'):    
        # print if the cell has text
        if cell.text and cell.text != ' ':
            textBuffer += (cell.text  + ' ').replace('\xa0', '')

textBuffer                

'APM2611 2023/0 1 assignment Closed 547463 2023-05-24        2 assignment Closed 547562 2023-06-15        3 assignment Closed 547640 2023-08-10        4 assignment Closed 741927 2023-09-21      APM2613 2023/0 1 quiz Closed 773178 2023-04-21        2 assignment Closed 773203 2023-05-30        3 assignment Closed 773262 2023-07-11        4 quiz Closed 773362 2023-08-23      COS2601 2023/0 1 quiz Closed 529201 2023-04-24        2 assignment Closed 531630 2023-06-12        3 assignment Closed 532530 2023-07-31        4 quiz Closed 532851 2023-09-26      COS2611 2023/0 1 quiz Closed 744381 2023-05-11        2 assignment Closed 744411 2023-07-19        3 quiz Closed 744419 2023-09-15      COS2614 2023/0 1 assignment Closed 322799 2023-05-26        2 quiz Closed 744077 2023-07-31        3 assignment Closed 744224 2023-08-31        4 quiz Closed 744246 2023-09-22        5 quiz Closed 744309 2023-10-20      COS2661 2023/0 1 quiz Closed 218852 2023-04-21        2 assignment Closed 220989 2023-06

In [151]:
data = []
rows = textBuffer.split('        ')
for row in rows:
    if len(row.split('      ')) > 1:
        data.append(row.split('      ')[0])
        data.append(row.split('      ')[1])
    else:
        data.append(row)
import pandas as pd
df = pd.DataFrame(columns=['Course', 'Term', 'Number', 'Type', 'Status', 'ID', 'Date'])
# iterate over the data and add to the dataframe
subject = ''
term = ''

for row in data:
    try:
        # if first character is a digit
        firstChar = row.split(' ')[0]
        if firstChar[0].isdigit():
            # parse 2 assignment Closed 547562 2023-06-15
            df = df.append({
                'Course': subject,
                'Term': term,
                'Number': row.split(' ')[0],
                'Type': row.split(' ')[1],
                'Status': row.split(' ')[2],
                'ID': row.split(' ')[3],
                'Date': row.split(' ')[4]
            }, ignore_index=True)
            
        else:
            # parse APM2611 2023/0 1 assignment Closed 547463 2023-05-24
            subject = row.split(' ')[0]
            term = row.split(' ')[1]
            df = df.append({
                'Course': row.split(' ')[0],
                'Term': row.split(' ')[1],
                'Number': row.split(' ')[2],
                'Type': row.split(' ')[3],
                'Status': row.split(' ')[4],
                'ID': row.split(' ')[5],
                'Date': row.split(' ')[6]
            }, ignore_index=True)
    except:
        pass

C:\Users\ADGSENPAI\AppData\Local\Temp\ipykernel_16948\3388798525.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({
C:\Users\ADGSENPAI\AppData\Local\Temp\ipykernel_16948\3388798525.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({
C:\Users\ADGSENPAI\AppData\Local\Temp\ipykernel_16948\3388798525.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({
C:\Users\ADGSENPAI\AppData\Local\Temp\ipykernel_16948\3388798525.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({
C:\Users\ADGSENPAI\AppData\Local\Temp\ipykernel_16948\3388798525.py:35: FutureWarning: The frame.append 

In [152]:
# sort the dates in ascending order
df = df.sort_values(by=['Date'], ascending=True)
df

,Course,Term,Number,Type,Status,ID,Date
23,MAT2611,2023/0,1,assignment,Closed,332663,2023-04-14
20,COS2661,2023/0,1,quiz,Closed,218852,2023-04-21
4,APM2613,2023/0,1,quiz,Closed,773178,2023-04-21
8,COS2601,2023/0,1,quiz,Closed,529201,2023-04-24
24,MAT2611,2023/0,2,assignment,Closed,332691,2023-04-28
35,MAT2613,2023/0,1,assignment,Closed,688316,2023-05-08
12,COS2611,2023/0,1,quiz,Closed,744381,2023-05-11
25,MAT2611,2023/0,3,assignment,Closed,332896,2023-05-15
38,MAT2615,2023/0,1,assignment,Closed,823949,2023-05-23
33,MAT2612,2023/0,1,assignment,Closed,839378,2023-05-23


In [153]:
# save all the data to a email calendar format
with open('calendar.ics', 'w') as f:
    f.write('BEGIN:VCALENDAR\n')
    f.write('VERSION:2.0\n')
    f.write('PRODID:-//hacksw/handcal//NONSGML v1.0//EN\n')
    f.write('CALSCALE:GREGORIAN\n')
    f.write('METHOD:PUBLISH\n')
    f.write('X-WR-CALNAME:Unisa Assignments\n')
    f.write('X-WR-TIMEZONE:Europe/Johannesburg\n')
    f.write('X-WR-CALDESC:Unisa Assignments\n')
    f.write('BEGIN:VTIMEZONE\n')
    f.write('TZID:Europe/Johannesburg\n')
    f.write('X-LIC-LOCATION:Europe/Johannesburg\n')
    f.write('BEGIN:STANDARD\n')
    f.write('TZOFFSETFROM:+0200\n')
    f.write('TZOFFSETTO:+0200\n')
    f.write('TZNAME:SAST\n')
    f.write('DTSTART:19700101T000000\n')
    f.write('END:STANDARD\n')
    f.write('END:VTIMEZONE\n')
    for index, row in df.iterrows():
        f.write('BEGIN:VEVENT\n')
        f.write('UID:unisa-assignment-' + row['ID'] + '\n')
        f.write('DTSTAMP:20200315T000000Z\n')
        f.write('DTSTART;TZID=Europe/Johannesburg:' + row['Date'].replace('-', '') + 'T000000\n')
        f.write('DTEND;TZID=Europe/Johannesburg:' + row['Date'].replace('-', '') + 'T000000\n')
        f.write('SUMMARY:' + row['Course'] + ' - ' + row['Type'] + ' - ' + row['Status'] + '\n')
        f.write('DESCRIPTION:' + row['Course'] + ' - ' + row['Type'] + ' - ' + row['Status'] + '\n')
        f.write('END:VEVENT\n')
    f.write('END:VCALENDAR\n')

# save the dataframe to a csv file
df.to_csv('unisa_assignments.csv', index=False)